# Introduction

In this tutorial, we will explain the basics of using Data Detective to perform a Data Investigation. In this tutorial, we will go through the steps of setting up and running a basic Data Detective Investigation, including: 
1. Configuring a dataset that works with Data Detective. 
2. Setting up a schema that defines the Data Detective investigation. 
3. Executing a data detective investigation. 
4. Summarizing results using the built-in Rank Aggregator


We will also include supplemental tutorials for some of Data Detective's more advanced features, including: 
- Extending the Data Detective investigation with custom validations
- Extending the transform library to map custom datatypes to supported datatypes


Let's get started!


In [ ]:
import os
import sys
import time
import torchvision.transforms as transforms

from typing import Dict

src_path = os.path.abspath(os.path.join(os.pardir, '.'))
if src_path not in sys.path:
    sys.path.insert(0, src_path)


from src.actions.action import RemoveTopKAnomalousSamplesAction
from src.aggregation.rankings import ResultAggregator, RankingAggregationMethod, ScoreAggregationMethod
from src.data_detective_engine import DataDetectiveEngine
from src.datasets.tutorial_dataset import TutorialDataset
from src.datasets.data_detective_dataset import dd_random_split

%pdb

# Dataset Construction

## Requirements for a Dataset

For a dataset to work within the Data Detective framework, it needs to satisfy the following requirements: 

1. It must override the `__getitem__` method that returns a dictionary mapping from each data column key to the data value. 
2. It must contain a `datatypes` method that returns a dictionary mapping from each data column key to the column's datatype. 

Let's examine what this looks like in practice.


## Dataset Implementation

In this tutorial, we will create a heterogeneous dataset that consists of the following items: 

- MNIST images
- MNIST labels
- 10-dimensional normal distribution (μ=0, σ=1)

The full dataset can be found under src/datasets/tutorial_dataset.


In [ ]:
dataset = TutorialDataset(
    root='./data/MNIST',
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor() 
    ])
)

In [ ]:
dataset.show_datapoint(0)

In [ ]:
dataset[0]["normal_vector_2"]

# Setting up the Data Object

The *data object* is a dictionary that consists of the preprocessed dataset and (optionally) its splits. In order to make use of split validation techniques such as distribution shift between splits, the data splits must be included. 

There are 3 basic categories of split objects that can be included in the data object:
1. **The Entire Set**. This set consists of the entire dataset being analyzed. This is held under the key `entire_set`.
2. **Unordered Split Sets**. These are splits that are order-invariant for the purposes of analysis. For example, when doing covariate analysis between splits, the order and names of the dataset does not need to be specified to compare all datasets to all other datasets. They are defined as a nested dictionary under the key `split_group_set` under the key `split_group_set`; see the example below for more details. 
3. **Ordered Splits**. These are splits that are order-dependent for the purposes of analysis. For example, when doing OOD detection, it is important to define a source dataset for training the OOD model and the target split for evaluation. 
   - Source/training splits are labeled as the `everything_but_inference_set`
   - Target/inference splits are labeled as the `inference_set`

The example below should clarify these distinctions.

In [ ]:
inference_size: int = 20
everything_but_inference_size: int = dataset.__len__() - inference_size
inference_dataset, everything_but_inference_dataset = dd_random_split(dataset, [inference_size, dataset.__len__() - inference_size])
    
train_size: int = int(0.6 * len(everything_but_inference_dataset))
val_size: int = int(0.2 * len(everything_but_inference_dataset))
test_size: int = len(everything_but_inference_dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = dd_random_split(everything_but_inference_dataset, [train_size, val_size, test_size])

data_object = {
    "entire_set": dataset,
    "everything_but_inference_set": everything_but_inference_dataset,
    "inference_set": inference_dataset,
    "train/val/test": {
        "training_set": train_dataset,
        "validation_set": val_dataset,
        "test_set": test_dataset,
    },
}

print(f"size of inference_dataset: {inference_dataset.__len__()}")
print(f"size of everything_but_inference_dataset: {everything_but_inference_dataset.__len__()}")
print(f"size of train_dataset: {train_dataset.__len__()}")
print(f"size of entire dataset: {dataset.__len__()}")
print(f"size of val_dataset: {val_dataset.__len__()}")
print(f"size of test_dataset: {test_dataset.__len__()}")

# Setting up a Validation Schema

## Specifying Validators and Options

The validation schema contains information about the types of checks that will be executed by the Data Detective Engine and the transforms that Data Detective will use. Before discussing the validation schema, it is important to define two key modular components that make up Data Detective investigations. Data Detective's functionality can be divided into a modular, implementation-heavy component referred to as a *validator method* and an easily toggleable component referred to as a *validator*.

A *validator method* performs a specific type of test for a data issue on a specific data type. These validator methods primarily consist of the code needed to take a dataset and run an evaluation on it that produces either a positive or negative result or a score. Some examples of validator methods include:
- Mann-Whitney U-Test to examine distribution shift between train/test splits on tabular data
- Kernel Conditional Independence (KCI) test for validating causal assumptions on vectorvalued
data
- Isolation forest trained/evaluated on image histograms for identifying anomalies in imaging
data

*Validations* are toggleable, datatype-agnostic collections of validator methods that are serially applied to the dataset. Each validator targets a single problem that may arise in data, including: 
- Shift between different data splits
- Outlier and anomaly detection
- Violation of parametric assumptions on the data
- Violation of expected casual structures / conditional independences in the 

In the validation schema, users only specify the validators that they would like to use. This abstracts away details concerning what methods should be used for which columns and simplifies the process for searching for a particular flavor of issues to a few lines of code. Below is the validation schema that we will use for the tutorial. 

In [ ]:
validation_schema : Dict = {
    "validators": {
        "unsupervised_anomaly_data_validator": {
            "validator_kwargs":{
                "should_return_model_instance": True
            }
        },
        # "unsupervised_multimodal_anomaly_data_validator": {},
        # "split_covariate_data_validator": {},
        # "ood_inference_data_validator": {}
    }
}

A few notes: 
- Each validator maps to an object that specifies additional options. For this tutorial, we will use the default settings, but these options include: 
  - special kwargs to include and pass to the validator methods
  - additional filtering regarding which columns the validator should be applied to
 
  
  
 

## Specifying Transforms

It may be the case that you are using a data modality that has little to no method infrastructure in Data Detective. The simplest way to make use of all of Data Detective's functionality is to use a transform that maps this data modality to a well-supported modality in Data Detective such as multidimensional data. In our example, we will be making use of a pretrained resnet50 backbone to map our MNIST images to 2048 dimensional vectors. This will allow us to make use of methods built for multidimensional data on our MNIST image representations.

In [ ]:
transform_schema : Dict = {
    "transforms": {
        "IMAGE": [{
            "name": "resnet50",
            "in_place": "False",
            "options": {},
        }],
    }
}
     
full_validation_schema: Dict = {
    **validation_schema, 
    **transform_schema
}

# Running the Data Detective Engine

Now that the full validation schema and data object are prepared, we are ready to run the Data Detective Engine.

In [ ]:
data_detective_engine = DataDetectiveEngine()

start_time = time.time()
results = data_detective_engine.validate_from_schema(full_validation_schema, data_object, run_concurrently=True)
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
results

Great! Let's start to look at and analyze the results we've collected.

# Aggregating + Examining Results using the Built-In Result Aggregator

In [ ]:
aggregator = ResultAggregator(results_object=results)
modal_rankings = aggregator.aggregate_results_modally("unsupervised_anomaly_data_validator", [RankingAggregationMethod.LOWEST_RANK, RankingAggregationMethod.HIGHEST_RANK, RankingAggregationMethod.ROUND_ROBIN], given_data_modality="resnet50_backbone_mnist_image")
total_rankings = aggregator.aggregate_results_multimodally("unsupervised_anomaly_data_validator", [RankingAggregationMethod.LOWEST_RANK, RankingAggregationMethod.HIGHEST_RANK, RankingAggregationMethod.ROUND_ROBIN, ScoreAggregationMethod.NORMALIZED_AVERAGE])
# rankings = total_rankings
rankings = modal_rankings

rankings

# Taking Action: Fixing our Dataset with Built-in Actions

In [ ]:
action = RemoveTopKAnomalousSamplesAction()
new_data_object = action.get_new_data_object(
    data_object,
    rankings, 
    "round_robin_agg_rank"
)

In [ ]:
for val in {'114', '286', '28', '81', '153', '333', '53', '453', '12', '10'}:
    dataset.show_datapoint(int(val))

# Wait, but Why: Occlusion-Oriented Interpretability

Lets look at a couple examples.

In [ ]:
INDICES_TO_EXAMINE = [98, 10, 12]

from src.transforms.transform_library import TRANSFORM_LIBRARY
from src.utils import OcclusionTransform, occlusion_interpretability

METHOD = 'cblof_anomaly_validator_method'

resnet = TRANSFORM_LIBRARY['resnet50']()
resnet.initialize_transform(transform_kwargs={})
print(resnet)

for idx in INDICES_TO_EXAMINE:
    sample = dataset[idx]
    img = sample['mnist_image']
    img = img.reshape((1, 28, 28))
    occ = OcclusionTransform(width=5)
    occed = occ(img, (15, 15))

    model_results, model = results['unsupervised_anomaly_data_validator'][METHOD]['resnet50_backbone_mnist_image_results']
    res_min, res_max = min(model_results.values()), max(model_results.values())
    interp_results = occlusion_interpretability(img, model, occ, (res_min, res_max))